In [43]:
import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf

from tensorflow import keras
from keras_tuner import RandomSearch
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential

In [44]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [45]:
with tf.device('/device:CPU:0'):

    # Open velocity data set
    ds = xr.open_dataset('/Users/smata/Downloads/processedData/velocityData.nc')

    # Reformat to Pandas dataframe
    df_u = ds['u'].to_dataframe().reset_index()
    df_v = ds['v'].to_dataframe().reset_index()

    df_u = df_u.pivot(index='time', columns='height', values='u')
    df_v = df_v.pivot(index='time', columns='height', values='v')

    df_u.columns = [f'u_{int(height)}m' for height in df_u.columns]
    df_v.columns = [f'v_{int(height)}m' for height in df_v.columns]

    df = pd.concat([df_u, df_v], axis=1)
    df.reset_index(drop=True, inplace=True)

    df.insert(0, 'L', ds.L.values)
    df.insert(0, 'TKE', ds.TKE.values)

    df.insert(0, 'hour_sin', np.sin(2 * np.pi * ds.hr_day.values / 24))
    df.insert(0, 'hour_cos', np.cos(2 * np.pi * ds.hr_day.values / 24))

    df.insert(0, 'day_sin', np.sin(2 * np.pi * ds.day_yr.values / 365))
    df.insert(0, 'day_cos', np.cos(2 * np.pi * ds.day_yr.values / 365))

    # Standardize data
    windCols = [col for col in df.columns if col.startswith('u_') or col.startswith('v_')]
    stabilityCols = ['TKE', 'L']

    wind_scaler = MinMaxScaler()
    df[windCols] = wind_scaler.fit_transform(df[windCols])

    stability_scaler = MinMaxScaler()
    df[stabilityCols] = stability_scaler.fit_transform(df[stabilityCols])

    # Define and extract input and output columns
    inputs = ['day_cos', 'day_sin', 'hour_cos', 'hour_sin', 'TKE', 'L', 'u_10m', 'v_10m']

    X = df[inputs].values
    y = df[windCols].values

    # Create input and output arrays
    split_index = int(0.8 * len(X))

    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    # Define the model-building function
    def build_model(hp):
        model = keras.Sequential()
        
        # Choose number of layers
        for i in range(hp.Int('num_layers', 1, 10)):
            model.add(keras.layers.Dense(
                units = hp.Int(f'units_{i}', min_value = 32, max_value = 512, step = 16),
                activation = 'relu'))
        
        # Output layer
        model.add(keras.layers.Dense(y_train.shape[1]))  # Output size should match the number of wind components
        
        # Compile model
        model.compile(
            optimizer = keras.optimizers.Adam(
                hp.Choice('learning_rate', values = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5])
            ),
            loss = 'mean_squared_error',
            metrics = ['mae'])
        
        return model

    # Initialize the Keras Tuner
    tuner = RandomSearch(
        build_model,
        objective    = 'val_mae',
        max_trials   = 100,
        executions_per_trial = 10,
        directory    = 'hyperparameter_tuning',
        project_name = 'wind_forecast_nn'
    )

    # Split the training data into training and validation sets
    X_train, X_val = X_train[:int(len(X_train) * 0.8)], X_train[int(len(X_train) * 0.8):]
    y_train, y_val = y_train[:int(len(y_train) * 0.8)], y_train[int(len(y_train) * 0.8):]

    # Perform hyperparameter tuning
    tuner.search(X_train, y_train, epochs = 100, validation_data = (X_val, y_val))

    # Get the optimal hyperparameters
    best_hyperparameters = tuner.get_best_hyperparameters(num_trials = 1)[0]

    # Print the best hyperparameters
    print("Optimal number of layers:", best_hyperparameters.get('num_layers'))
    print("Optimal learning rate:", best_hyperparameters.get('learning_rate'))
    for i in range(best_hyperparameters.get('num_layers')):
        print(f"Optimal units in layer {i}: {best_hyperparameters.get(f'units_{i}')}")


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2                 |2                 |num_layers
416               |416               |units_0
0.01              |0.01              |learning_rate

Epoch 1/50
144/144 [==============================] - 0s 848us/step - loss: 0.0190 - mae: 0.0936 - val_loss: 0.0072 - val_mae: 0.0667
Epoch 2/50
144/144 [==============================] - 0s 602us/step - loss: 0.0062 - mae: 0.0597 - val_loss: 0.0060 - val_mae: 0.0591
Epoch 3/50
144/144 [==============================] - 0s 559us/step - loss: 0.0055 - mae: 0.0565 - val_loss: 0.0066 - val_mae: 0.0620
Epoch 4/50
144/144 [==============================] - 0s 550us/step - loss: 0.0055 - mae: 0.0560 - val_loss: 0.0073 - val_mae: 0.0677
Epoch 5/50
144/144 [==============================] - 0s 546us/step - loss: 0.0053 - mae: 0.0547 - val_loss: 0.0066 - val_mae: 0.0623
Epoch 6/50
144/144 [==============================] - 0s 596us/step - loss: 0.0052 - mae: 0.0544 - val

KeyboardInterrupt: 